<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>
# <a id="csid" style="text-decoration:none;">HPE CSI Driver Data Fabric for Kubernetes</a>
![](imgs/hpedevlogo.svg)
<img src="imgs/mapr-ar21.gif" alt="csi logo" style="float: right;">
<img src="imgs/csilogo1.png" alt="csi logo" style="float: right;">
## How to deploy SQL Server 2019 with an Operator on Kubernetes Cluster

Emmanuel COLUSSI 
<br>
<br>
<br>
<br>

## Introduction
Kubernetes, or k8s (k, 8 characters, s...get it?), or “kube” if you’re into brevity, is an open source platform that automates Linux container operations. It eliminates many of the manual processes involved in deploying and scaling containerized applications. In other words, you can cluster together groups of hosts running Linux containers, and Kubernetes helps you easily and efficiently manage those clusters. These clusters can span hosts across public, private, or hybrid clouds. 

<img src="imgs/k8s2.png" alt="Kubernetes Architecture" >

A Container Storage Interface (CSI) Driver for Kubernetes.The HPE MapR CSI Driver for Kubernetes allows you to use a Container Storage Provider to perform data management operations on storage resources.<br><br>
<img src="imgs/maprcsi.png" alt="csi logo" >


<p>MapR is providing this capability using a standard called the Container Storage Interface (CSI). This community-driven standard was recently established as a platform-independent way to provide persistent data storage in Container Orchestrator Systems (COs), such as Kubernetes, Mesos, and Cloud Foundry. This new capability maintains feature parity with the FlexVolume plugin that MapR released for Kubernetes last year and it aligns MapR with the priorities of CO communities where CSI is seeing high adoption and support.</p>

<p>MapR's storage plugin for Kubernetes provides the following capabilities to applications containers:</p>
<ul>
    <li>Containers can mount MapR volumes as a POSIX file system.</li>
    <li>Containers can create and delete MapR volumes.</li>
    <li>Containers can create MapR volume snapshots for point-in-time backups.</li>

</ul>

## Deploying to Kubernetes
> This guide is primarily written to accommodate installation on upstream Kubernetes. Installation steps may vary for different distributions of Kubernetes.<br>


> In this lab we don’t need to install the HPE Data Fabric cluster is preinstalled as well as the csi plugin driver for MapR.
For installation information, see Installing and Uninstalling the MapR Container Storage Interface (CSI) Storage Plugin : https://docs.datafabric.hpe.com/61/CSIdriver/csi_installation.html<br>

> Installing the MapR Container Storage Interface (CSI) Storage Plugin is easy, just execute the following command :<br>
> kubectl create -f https://raw.githubusercontent.com/mapr/mapr-csi/v1.2.0/deploy/kubernetes/csi-maprkdf-v1.2.0.yaml



Show is the MapR Container Storage Interface Storage Plugin driver is installed

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl get pods --all-namespaces -o wide |grep mapr-csi"

</br>
<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>

> To leverage the MapR file system with a Kubernetes cluster, you can create a PersistentVolume in Kubernetes.
> The following diagram shows the two ways in which the PersistentVolume can be provisioned for the POSIX client.

<img src="imgs/StaticVsDynamicProvisioning_rebrand.png" alt="Mapr" style="display: block;margin-left: auto;margin-right: auto;width:40%;">

<br>
<br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;The first step is to create Rest secret for dynamic volume provisioning with your platform details.<br></br>
<p>Kubernetes Secrets enable you to inject sensitive data into a pod.For dynamic provisioning, you must use a Secret to pass the user name and password of a MapR user to the provisioner. This user must have privileges to create and delete a MapR volume. The credentials allow the provisioner to make REST calls to the MapR webserver.</p>
<p>Replace the password string <b>'YWRtaW4='</b> with a base64 encoded version of mapr user password .</p>
<br>

<link rel="stylesheet" type="text/css" href="css/bootstrap.min.css">
<link rel="stylesheet" type="text/css" href="css/fontawesome.min.css">

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-file-alt"></i>&nbsp;&nbsp;secret file example : secure_rest_ticket.yaml</p>
 
</div>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;You should now see the secret in the mapr-student4 namespace:</p>
    <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i> Run the following commands in the next Cell with button <i class="fas fa-play"></i> : </p>
    <p class="mb-0"><i class="fas fa-hand-point-right"></i>&nbsp;&nbsp;create a namespace for the Demo </p> 
    <p class="mb-0"><i class="fas fa-hand-point-right"></i>&nbsp;&nbsp;create a secret </p> 
</div>

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/create_namespace.yaml"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/secure_rest_ticket.yaml -n mapr-$JUPYTERHUB_USER"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl -n mapr-$JUPYTERHUB_USER get secret/mapr-provisioner-secrets"

<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;The second step is to create a secret for MapR ticket with your platform details.<br></br>
<p>For static and dynamic provisioning, you must specify a Secret, which is the base64 representation of the ticket, to enable the POSIX client to communicate with a secure MapR cluster. The ticket for the POSIX client can be generated on the MapR cluster using the maprlogin utility.</p>
<p>The steps to generate the ticket or retrieve the value of the existing ticket are as follows : </p>
<ul>
    <li>Logon MapR Cluster, and locate the ticket file using "maprlogin print" or generate a new ticket file using "maprlogin password" :<br> 
For example, here we are using "mapr" user ticket file located at /tmp/maprticket_5000.</li>
    <li>Convert the ticket into base64 representation and save the output : <br>
        cat /tmp/maprticket_5000 | base64</li>
    <li> Assign the value of the converted ticket to <b>CONTAINER_TICKET</b></li>
</ul>    

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-file-alt"></i>&nbsp;&nbsp;secret file example : secure_ticket.yaml</p>
 
</div>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  <p class="mb-0"><i class="fas fa-hand-point-right"></i>&nbsp;&nbsp;create a secret </p> 
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;You should now see the secret for MapR ticket in the mapr-student4 namespace:</p>
   
</div>

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/secure_tickets.yaml -n mapr-$JUPYTERHUB_USER"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl -n mapr-$JUPYTERHUB_USER get secret/mapr-provisioner-secrets"

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

HPE CSI Driver for MapR is installed ,the next step is to create a Storage Class which will be used to provision persistent volumes

 
 <a href="HPEMAPR-SQL-SC.ipynb#sc" target="New" title="Next: Create Storage Class"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a> 
 <a href="HPEMAPR-SQL-Intro.ipynb#intro" target="New" title="Back: About This demontration"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a> 
 
 




<p style="display: block;margin-left: auto;margin-right: auto;width:10%;" ><a href="#csid" title="TOP"><i class="fas fa-2x fa-chevron-circle-up" style="color:#FFAD33;"></i></a></p>